https://github.com/adeshpande3/LSTM-Sentiment-Analysis/blob/master/Oriole%20LSTM.ipynb  
http://colah.github.io/posts/2015-08-Understanding-LSTMs/  
http://karpathy.github.io/2015/05/21/rnn-effectiveness/  


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys
import os
import math
import spacy
sys.path.append("../")
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [2]:
import matplotlib.pyplot as plt
plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

from dhira.data.data_manager import DataManager
from dhira.data.embedding_manager import EmbeddingManager

from dhira.data.dataset.movie_review import MovieReview
from dhira.data.features.movie_review_feature import MovieReviewFeature


In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
mr = MovieReview()
data_manager = DataManager(dataset=mr, nlp=nlp)
get_train_data_gen, train_data_size = data_manager.get_train_data()
get_val_data_gen, val_data_size = data_manager.get_validation_data()

INFO:dhira.data.dataset.internal.text:Reusing the pickle file movie-review-data-data_indexer.p.
INFO:dhira.data.dataset.internal.dataset_base:Reusing the pickle file movie-review-data-train.p.
INFO:dhira.data.dataset.internal.text:Trying to load prefitted data_indexer... movie-review-data-data_indexer.p
INFO:dhira.data.dataset.internal.text:Instance max lengths {'num_sentence_words': 63}
INFO:dhira.data.dataset.internal.text:Padding lengths to length: {'num_sentence_words': 59}
INFO:dhira.data.dataset.internal.dataset_base:movie-review-data-train.p already exists
INFO:dhira.data.dataset.internal.dataset_base:Reusing the pickle file None.
INFO:dhira.data.dataset.internal.text:Instance max lengths {'num_sentence_words': 54}
INFO:dhira.data.dataset.internal.text:Padding lengths to length: {'num_sentence_words': 59}
INFO:dhira.data.dataset.internal.dataset_base:movie-review-data-val.p already exists


In [5]:
embedding_matrix = data_manager.embedding_matrix
embedding_matrix.shape

(16691, 300)

In [6]:
batch_size = 128
num_epochs = 10
num_train_steps_per_epoch = int(math.ceil(train_data_size / batch_size))
num_val_steps = int(math.ceil(val_data_size / batch_size))
print('Number of train steps {} and validation steps {} for batch size {}'.format(num_train_steps_per_epoch, 
                                                                                  num_val_steps,
                                                                                  batch_size))

Number of train steps 67 and validation steps 9 for batch size 128


In [7]:
log_period = 10
val_period = num_train_steps_per_epoch//1.8
save_period = num_train_steps_per_epoch//1.8
val_period

37.0

In [8]:
from dhira.tf.models.lstm.sentiment_analysis import LSTMSentimentAnalysis
model = LSTMSentimentAnalysis(save_dir='../models/',
                run_id=1,
             word_vocab_size=embedding_matrix.shape[0], 
             word_embedding_dim=300, 
             word_embedding_matrix=embedding_matrix, 
             fine_tune_embeddings=True, 
             sequence_legth=59,
             rnn_output_mode='last',
             output_keep_prob=0.5,
             rnn_hidden_size=256,
             batch_size=batch_size,
             num_lstm_layers=1)


INFO:dhira.tf.models.internal.base_tf_model:Writing logs to /home/mageswarand/.dhira/logs/LSTMSentimentAnalysis/011502941818/
INFO:dhira.tf.models.internal.base_tf_model:log path /home/mageswarand/.dhira/logs/LSTMSentimentAnalysis/011502941818/ does not exist, creating it


In [ ]:
model.compile()

INFO:dhira.tf.models.internal.base_tf_model:Writing params to /home/mageswarand/.dhira/logs/LSTMSentimentAnalysis/011502941818/LSTMSentimentAnalysisparams.json
INFO:dhira.tf.models.internal.base_tf_model:Building graph...
INFO:dhira.tf.models.lstm.sentiment_analysis:word_emb_mat: ------> <tf.Variable 'embeddings/embedding_var/word_emb_mat:0' shape=(16691, 300) dtype=float32_ref>
INFO:dhira.tf.models.lstm.sentiment_analysis:word_embedded_sentence: ------> Tensor("embeddings_1/word_embeddings/embedding_lookup:0", shape=(?, 59, 300), dtype=float32, device=/device:CPU:0)
INFO:dhira.tf.models.lstm.sentiment_analysis:lstm: ------> <tensorflow.python.ops.rnn_cell_impl.LSTMCell object at 0x7f5c0f26eb38>
INFO:dhira.tf.models.lstm.sentiment_analysis:cell: ------> <tensorflow.python.ops.rnn_cell_impl.MultiRNNCell object at 0x7f5c0f26e9e8>
INFO:dhira.tf.models.lstm.sentiment_analysis:initial_state: ------> (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/SwitchableDropoutWrapperZeroState/LSTMCe

In [ ]:

model.train(get_train_feature_generator=get_train_data_gen,
            get_val_feature_generator=get_val_data_gen,
            batch_size=batch_size,
            num_train_steps_per_epoch=num_train_steps_per_epoch,
            num_epochs=num_epochs,
            num_val_steps=num_val_steps,
            log_period=log_period,
            val_period=val_period,
            save_period=save_period,
            patience=2)

Writing to /home/mageswarand/.dhira/logs/LSTMSentimentAnalysis/011502941818

--------------------------------------------------

tensorboard --logdir  /home/mageswarand/.dhira/logs/LSTMSentimentAnalysis/011502941818

tensorboard --logdir  /opt/dhira/models/LSTMSentimentAnalysis/01/checkpoints --port 6007
--------------------------------------------------


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


INFO:dhira.tf.models.internal.base_tf_model:Saving final model @ /opt/dhira/models/LSTMSentimentAnalysis/01/LSTMSentimentAnalysis-37
INFO:dhira.tf.models.internal.base_tf_model:Saving final model @ /opt/dhira/models/LSTMSentimentAnalysis/01/LSTMSentimentAnalysis-67
Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


INFO:dhira.tf.models.internal.base_tf_model:Saving final model @ /opt/dhira/models/LSTMSentimentAnalysis/01/LSTMSentimentAnalysis-74
Widget Javascript not detected.  It may not be installed or enabled properly.


INFO:dhira.tf.models.internal.base_tf_model:Saving final model @ /opt/dhira/models/LSTMSentimentAnalysis/01/LSTMSentimentAnalysis-111
INFO:dhira.tf.models.internal.base_tf_model:Saving final model @ /opt/dhira/models/LSTMSentimentAnalysis/01/LSTMSentimentAnalysis-134
Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [ ]:
def get_mood(comment):
    logger.setLevel(level=40)
    single_feature = mr.custom_input(comment)
    single_feature_batch = DataManager.to_batch(single_feature)
    pred = model.predict(single_feature_batch)
    if pred[0][0] > pred[0][1]: 
        print('Positive Review', pred)
    else:
        print('Negative Review', pred)
    logger.setLevel(level=10)

In [ ]:
get_mood('that was a awesome moview')

In [ ]:
get_mood('that was not a bad movie')